In [11]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
import importlib

import trainer, data, models
from data import gen_lin_data
from trainer import ModelingDataset, Trainer, Hyperparameters
from models import LinearRegression

In [12]:
importlib.reload(trainer)
importlib.reload(data)
importlib.reload(models)

<module 'models' from '/Users/ralph/projects/d2dl/models.py'>

In [13]:
train_X, train_y = gen_lin_data(torch.randn(3), 1.0, 8000)
val_X, val_y = gen_lin_data(torch.randn(3), 1.0, 2000)

In [14]:
train_dataset = TensorDataset(train_X, train_y)
val_dataset = TensorDataset(val_X, val_y)

In [38]:
linreg_trainer = Trainer(
    model=LinearRegression(),
    dataset=ModelingDataset(train_dataset, val_dataset),
    loss=nn.MSELoss,
    opt=torch.optim.SGD,
    hyperparameters=Hyperparameters(
        general=dict(
            batch_size=100,
            num_epochs=1000
        ),
        opt=dict(
            lr=1e-3,
            weight_decay=0
        )
    ),
)

/Users/ralph/.venvs/d2dl/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [39]:
linreg_trainer.train(10)

In [28]:
list(linreg_trainer.model.parameters())

[Parameter containing:
 tensor([[0.1630, 0.1500, 0.2019]], requires_grad=True),
 Parameter containing:
 tensor([0.3350], requires_grad=True)]